<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/kgcnn_jarvis_leaderboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install jarvis_leaderboard

In [1]:
pip install  jarvis_leaderboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pandas-1.2.4-cp39-cp39-manylinux1_x86_64.whl (9.7 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.1
    Uninstalling pandas-2.0.1:
      Successfully uninstalled pandas-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.18 requires pandas>=1.3.0, but you have pandas 1.2.4 which is incompatible.
xarray 2022.12.0 requires pandas>=1.3, but you have pandas 1.2.4 which is incompatible.
plotnine 0.10.1 requires pandas>=1.3.5, but you have pandas 1.2.4 which is incompatible.
mizani 0.8.1 requires pandas>=1.3.5, but you have pandas 1.2.4 which is incompatible.
google-colab 1.0.0 requires pandas~=1.5.3, but you have pandas 1.2.4 which is incompatible.
arviz 0.15.1 requires pandas>=1.3.0, but you have pandas 1.2.

Install kgcnn

In [2]:
import os
if not os.path.exists('gcnn_keras'):
  !git clone https://github.com/aimat-lab/gcnn_keras.git

In [3]:
!pip install -e ./gcnn_keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gcnn_keras
  Preparing metadata (setup.py) ... done
  Using cached pandas-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
      Successfully uninstalled pandas-1.2.4
  Attempting uninstall: kgcnn
    Found existing installation: kgcnn 3.0.1
    Can't uninstall 'kgcnn'. No files were found to uninstall.
  Running setup.py develop for kgcnn
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jarvis-leaderboard 2023.4.24 requires pandas==1.2.4, but you have pandas 2.0.1 which is incompatible.
google-colab 1.0.0 requires pandas~=1.5.3, but you have pandas 2.0.1 which is incompatible.


might have to restart runtime here in case you cant import kgcnn

In [4]:
import kgcnn

Files will be saved in the output_path i.e. 'Out' directory

In [5]:
!jarvis_populate_data.py --benchmark_file SinglePropertyPrediction-test-exfoliation_energy-dft_3d-AI-mae --output_path=Out

benchmark_file SinglePropertyPrediction-test-exfoliation_energy-dft_3d-AI-mae
dataset dft_3d
output_path Out
property exfoliation_energy
method AI
task SinglePropertyPrediction
id_tag jid
dataset file to be used /usr/local/lib/python3.9/dist-packages/jarvis_leaderboard/dataset/AI/SinglePropertyPrediction/dft_3d_exfoliation_energy.json.zip
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
number of training samples 650
number of validation samples 81
number of test samples 81


In [6]:
import pandas as pd
import os
from jarvis.core.atoms import Atoms
from kgcnn.data.crystal import CrystalDataset
import numpy as np

def prepare_data(dirname='exfoliation_en',n_train=650,n_val=81,n_test=81,populated_data_path='Out'):

    id_prop_path=os.path.join(populated_data_path,'id_prop.csv')
    df=pd.read_csv(id_prop_path,header=None)
    df.columns=['id','target']
    df_train = df[0:n_train]
    df_val = df[n_train:n_train+n_val]
    df_test = df[n_train+n_val:]
   
    #train
    train_dir = dirname+'_train'
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    train_cif = os.path.join(train_dir,'CifFiles')
    if not os.path.exists(train_cif):
        os.makedirs(train_cif)
    csv_file = train_dir+'/data.csv'
    print ('train_dir',train_dir)
    f=open(csv_file, "w")
    f.write("file_name,index,label\n")
    targets = []
    for i,ii in df_train.iterrows():
        pos_path=os.path.join(populated_data_path,ii['id'])
        atoms=Atoms.from_poscar(pos_path)
        pmg = (atoms.pymatgen_converter())
        targets.append(ii['target'])
        fname="file_"+str(ii['id'])+".cif"
        #fname="file_"+str(i)+".cif"
        cif_name = os.path.join(train_cif,fname)
        pmg.to(filename=cif_name, fmt="cif")
        line=fname+','+str(i)+','+str(ii['target'])+'\n'
        f.write(line)
    f.close()
    train_dataset = CrystalDataset(
        data_directory=train_dir, 
        dataset_name=train_dir, 
        file_name="data.csv", 
        file_directory="CifFiles"
    )
    train_dataset.prepare_data(file_column_name="file_name", overwrite=True)
    train_dataset.read_in_memory(label_column_name="label")
    train_labels = np.expand_dims(train_dataset.get("graph_labels"), axis=-1)
    

    
    #val
    val_dir = dirname+'_val'
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)
    val_cif = os.path.join(val_dir,'CifFiles')
    if not os.path.exists(val_cif):
        os.makedirs(val_cif)
    csv_file = val_dir+'/data.csv'
    print ('val_dir',val_dir)
    f=open(csv_file, "w")
    f.write("file_name,index,label\n")
    targets = []
    for i,ii in df_val.iterrows():
        pos_path=os.path.join(populated_data_path,ii['id'])
        atoms=Atoms.from_poscar(pos_path)
        pmg = (atoms.pymatgen_converter())
        targets.append(ii['target'])
        fname="file_"+str(ii['id'])+".cif"
        #fname="file_"+str(i)+".cif"
        cif_name = os.path.join(val_cif,fname)
        pmg.to(filename=cif_name, fmt="cif")
        line=fname+','+str(i)+','+str(ii['target'])+'\n'
        f.write(line)
    f.close()
    val_dataset = CrystalDataset(
        data_directory=val_dir, 
        dataset_name=val_dir, 
        file_name="data.csv", 
        file_directory="CifFiles"
    )
    val_dataset.prepare_data(file_column_name="file_name", overwrite=True)
    val_dataset.read_in_memory(label_column_name="label")
    val_labels = np.expand_dims(val_dataset.get("graph_labels"), axis=-1)
    
    

    #test
    test_dir = dirname+'_test'
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
    test_cif = os.path.join(test_dir,'CifFiles')
    if not os.path.exists(test_cif):
        os.makedirs(test_cif)
    csv_file = test_dir+'/data.csv'
    print ('test_dir',test_dir)
    f=open(csv_file, "w")
    f.write("file_name,index,label\n")
    targets = []
    for i,ii in df_test.iterrows():
        pos_path=os.path.join(populated_data_path,ii['id'])
        atoms=Atoms.from_poscar(pos_path)
        pmg = (atoms.pymatgen_converter())
        targets.append(ii['target'])
        fname="file_"+str(ii['id'])+".cif"
        #fname="file_"+str(i)+".cif"
        cif_name = os.path.join(test_cif,fname)
        pmg.to(filename=cif_name, fmt="cif")
        line=fname+','+str(i)+','+str(ii['target'])+'\n'
        f.write(line)
    f.close()
    test_dataset = CrystalDataset(
        data_directory=test_dir, 
        dataset_name=test_dir, 
        file_name="data.csv", 
        file_directory="CifFiles"
    )
    test_dataset.prepare_data(file_column_name="file_name", overwrite=True)
    test_dataset.read_in_memory(label_column_name="label")
    test_labels = np.expand_dims(train_dataset.get("graph_labels"), axis=-1)
    
    
    
    return train_dataset,train_labels,val_dataset,val_labels,test_dataset,test_labels

train_dataset,train_labels,val_dataset,val_labels,test_dataset,test_labels = prepare_data(n_train=650,n_val=81,n_test=81,populated_data_path='Out')



train_dir exfoliation_en_train


INFO:kgcnn.data.exfoliation_en_train:Searching for structure files in 'exfoliation_en_train/CifFiles'
INFO:kgcnn.data.exfoliation_en_train:Read 650 single files.
INFO:kgcnn.data.exfoliation_en_train:... Read .cif file 0 from 650
/usr/local/lib/python3.9/dist-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
INFO:kgcnn.data.exfoliation_en_train:Exporting as dict for pymatgen ...
INFO:kgcnn.data.exfoliation_en_train:Saving structures as .json ...
INFO:kgcnn.data.exfoliation_en_train:Making node features from structure...
INFO:kgcnn.data.exfoliation_en_train:Reading structures from .json ...
INFO:kgcnn.data.exfoliation_en_train: ... read structures 0 from 650


val_dir exfoliation_en_val


INFO:kgcnn.data.exfoliation_en_val:Searching for structure files in 'exfoliation_en_val/CifFiles'
INFO:kgcnn.data.exfoliation_en_val:Read 81 single files.
INFO:kgcnn.data.exfoliation_en_val:... Read .cif file 0 from 81
/usr/local/lib/python3.9/dist-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
INFO:kgcnn.data.exfoliation_en_val:Exporting as dict for pymatgen ...
INFO:kgcnn.data.exfoliation_en_val:Saving structures as .json ...
INFO:kgcnn.data.exfoliation_en_val:Making node features from structure...
INFO:kgcnn.data.exfoliation_en_val:Reading structures from .json ...
INFO:kgcnn.data.exfoliation_en_val: ... read structures 0 from 81
INFO:kgcnn.data.exfoliation_en_test:Searching for structure files in 'exfoliation_en_test/CifFiles'
INFO:kgcnn.data.exfoliation_en_test:Read 81 s

test_dir exfoliation_en_test


INFO:kgcnn.data.exfoliation_en_test:Exporting as dict for pymatgen ...
INFO:kgcnn.data.exfoliation_en_test:Saving structures as .json ...
INFO:kgcnn.data.exfoliation_en_test:Making node features from structure...
INFO:kgcnn.data.exfoliation_en_test:Reading structures from .json ...
INFO:kgcnn.data.exfoliation_en_test: ... read structures 0 from 81


In [25]:
import pandas as pd
import os
from jarvis.core.atoms import Atoms
from kgcnn.data.crystal import CrystalDataset
import numpy as np

def prepare_data(dirname='exfoliation_en',populated_data_path='Out'):

    id_prop_path=os.path.join(populated_data_path,'id_prop.csv')
    df=pd.read_csv(id_prop_path,header=None)
    df.columns=['id','target']

   
    #train
    train_dir = dirname+'_train'
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    train_cif = os.path.join(train_dir,'CifFiles')
    if not os.path.exists(train_cif):
        os.makedirs(train_cif)
    csv_file = train_dir+'/data.csv'
    print ('train_dir',train_dir)
    f=open(csv_file, "w")
    f.write("file_name,index,label\n")
    targets = []
    for i,ii in df.iterrows():
        pos_path=os.path.join(populated_data_path,ii['id'])
        atoms=Atoms.from_poscar(pos_path)
        pmg = (atoms.pymatgen_converter())
        targets.append(ii['target'])
        fname="file_"+str(ii['id'])+".cif"
        #fname="file_"+str(i)+".cif"
        cif_name = os.path.join(train_cif,fname)
        pmg.to(filename=cif_name, fmt="cif")
        line=fname+','+str(i)+','+str(ii['target'])+'\n'
        f.write(line)
    f.close()
    dataset = CrystalDataset(
        data_directory=train_dir, 
        dataset_name=train_dir, 
        file_name="data.csv", 
        file_directory="CifFiles"
    )
    dataset.prepare_data(file_column_name="file_name", overwrite=True)
    dataset.read_in_memory(label_column_name="label")
    labels = np.expand_dims(train_dataset.get("graph_labels"), axis=-1)
    
  
    

    
    
    
    return dataset,labels

dataset,labels = prepare_data(populated_data_path='Out')



train_dir exfoliation_en_train


INFO:kgcnn.data.exfoliation_en_train:Searching for structure files in 'exfoliation_en_train/CifFiles'
INFO:kgcnn.data.exfoliation_en_train:Read 812 single files.
INFO:kgcnn.data.exfoliation_en_train:... Read .cif file 0 from 812
/usr/local/lib/python3.9/dist-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
INFO:kgcnn.data.exfoliation_en_train:Exporting as dict for pymatgen ...
INFO:kgcnn.data.exfoliation_en_train:Saving structures as .json ...
INFO:kgcnn.data.exfoliation_en_train:Making node features from structure...
INFO:kgcnn.data.exfoliation_en_train:Reading structures from .json ...
INFO:kgcnn.data.exfoliation_en_train: ... read structures 0 from 812


In [26]:
n_train=650
n_val=81
n_test=81
train_index=np.arange(0,n_train)
val_index=np.arange(n_train,n_train+n_val)
test_index=np.arange(n_train+n_val,n_train+n_val+n_test)

In [32]:
x_train,y_train=dataset[train_index], labels[train_index]
# x_val, y_val = dataset[val_index].tensor(model_config["inputs"]), labels[val_index]
# x_test, y_test = dataset[test_index].tensor(model_config["inputs"]), labels[test_index]

In [35]:
model.fit(
    x_train.tensor(model_config["inputs"]),y_train,
    shuffle=True,
    batch_size=3,
    epochs=20,
    verbose=2,
)

TypeError: ignored

In [31]:
dataset[train_index].tensor(model_config["inputs"])

TypeError: ignored

In [20]:
from kgcnn.literature.Schnet import make_crystal_model
from tensorflow.keras.optimizers import Adam
ragged=True
model_config = {
    "name": "Schnet",
    "inputs": [
        {'shape': (None,), 'name': "node_number", 'dtype': 'float32', 'ragged': ragged},
        {'shape': (None, 3), 'name': "node_coordinates", 'dtype': 'float32', 'ragged': ragged},
        {'shape': (None, 2), 'name': "range_indices", 'dtype': 'int64', 'ragged': ragged},
        {'shape': (None, 3), 'name': "range_image", 'dtype': 'int64', 'ragged': ragged},
        {'shape': (3, 3), 'name': "graph_lattice", 'dtype': 'float32', 'ragged': False}
    ],
    "input_embedding": {
        "node": {"input_dim": 95, "output_dim": 64}
    },
    "interaction_args": {
        "units": 128, "use_bias": True, "activation": "kgcnn>shifted_softplus", "cfconv_pool": "sum"
    },
    "node_pooling_args": {"pooling_method": "mean"},
    "depth": 4,
    "gauss_args": {"bins": 25, "distance": 5, "offset": 0.0, "sigma": 0.4}, "verbose": 10,
    "last_mlp": {"use_bias": [True, True, True], "units": [128, 64, 1],
                 "activation": ['kgcnn>shifted_softplus', 'kgcnn>shifted_softplus', 'linear']},
    "output_embedding": "graph",
    "use_output_mlp": False,
    "output_mlp": None,  # Last MLP sets output dimension if None.
}
model = make_crystal_model(**model_config)
model.compile(
    loss="mean_absolute_error",
    optimizer=Adam(learning_rate=1e-04),
    metrics=["mean_absolute_error"],
)

model.fit(
    train_dataset.tensor(model_config["inputs"]),train_labels,
    shuffle=False,
    batch_size=10,
    epochs=20,
    verbose=2,
)

INFO:kgcnn.model.utils:Updated model kwargs:
INFO:kgcnn.model.utils:{'name': 'Schnet', 'inputs': [{'shape': (None,), 'name': 'node_number', 'dtype': 'float32', 'ragged': True}, {'shape': (None, 3), 'name': 'node_coordinates', 'dtype': 'float32', 'ragged': True}, {'shape': (None, 2), 'name': 'range_indices', 'dtype': 'int64', 'ragged': True}, {'shape': (None, 3), 'name': 'range_image', 'dtype': 'int64', 'ragged': True}, {'shape': (3, 3), 'name': 'graph_lattice', 'dtype': 'float32', 'ragged': False}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}}, 'make_distance': True, 'expand_distance': True, 'interaction_args': {'units': 128, 'use_bias': True, 'activation': 'kgcnn>shifted_softplus', 'cfconv_pool': 'sum'}, 'node_pooling_args': {'pooling_method': 'mean'}, 'depth': 4, 'gauss_args': {'bins': 25, 'distance': 5, 'offset': 0.0, 'sigma': 0.4}, 'verbose': 10, 'last_mlp': {'use_bias': [True, True, True], 'units': [128, 64, 1], 'activation': ['kgcnn>shifted_softplus', 'kgcnn>s

TypeError: ignored

In [11]:
train_dataset[0]

{'graph_labels': array(124.8),
 'node_coordinates': array([[3.27151598e-16, 2.03436970e+00, 4.43930931e+00],
        [2.03436970e+00, 0.00000000e+00, 2.45201648e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.03436970e+00, 2.03436970e+00, 2.49138434e-16],
        [2.03436970e+00, 0.00000000e+00, 5.40683277e+00],
        [3.27151598e-16, 2.03436970e+00, 1.48449302e+00]]),
 'node_frac_coordinates': array([[0.        , 0.5       , 0.64418799],
        [0.5       , 0.        , 0.35581201],
        [0.        , 0.        , 0.        ],
        [0.5       , 0.5       , 0.        ],
        [0.5       , 0.        , 0.78458528],
        [0.        , 0.5       , 0.21541472]]),
 'graph_lattice': array([[4.06873939e+00, 0.00000000e+00, 2.49138434e-16],
        [6.54303197e-16, 4.06873939e+00, 2.49138434e-16],
        [0.00000000e+00, 0.00000000e+00, 6.89132579e+00]]),
 'abc': array([4.06873939, 4.06873939, 6.89132579]),
 'charge': array([0.]),
 'volume': array([114.0834

In [17]:
train_dataset.clean(model_config["inputs"])


INFO:kgcnn.data.exfoliation_en_train:No invalid graphs for assigned properties found.


array([], dtype=int64)

In [18]:
train_dataset.tensor(model_config["inputs"])

TypeError: ignored